In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [3]:
df = pd.read_csv('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/2021년 기사(완료).csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df.reset_index(drop=True, inplace=True)
df

,date,url,category,tokenize_title,tokenize_content
0,2021-01-01T03:37:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['네이버', '카카오', '추천', '뉴스', '사실', '여부', '보다', '...","['네이버', '자체', '개발', '콘텐츠', '추천', '시스템', '어스', ..."
1,2021-01-01T08:00:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['주식', '부호', '클럽', '김범수', '카카오', '의장', '순위', '...","['코스피', '사상', '최고', '마감', '주식', '부자', '늘어', '고..."
2,2021-01-01T13:04:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['클럽', '카카오', '김범수', '늘', '고', '아모레', '경배', '폭락']","['코스피', '지수', '사상', '최고', '치', '마감', '하면서', '이..."
3,2021-01-01T13:21:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['테크', '체인', '네이버', '보다', '카카오', '일상', '바꿀', '...","['네이버', '카카오', '응답', '우리', '일상', '바꿀', '기대', '..."
4,2021-01-01T17:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['작년', '매출', '신기록', '쓴', '카카오', '엔씨', '올해', '좋다']","['증권사', '망치', '분석', '최대', '실적', '예고', '카카오', '..."
...,...,...,...,...,...
8422,2021-12-30T14:49:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['뉴스', '픽', '카카오', '지그재그', '패션', '임박']","['카카오', '온라인', '패션', '플랫폼', '지그재그', '자체', '브랜드..."
8423,2021-12-30T16:33:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['상생', '외친', '카카오', '모빌리티', '일', '성', '아닌', '지...","['업계', '의견', '수렴', '해', '장기', '상생', '구현', '목표'..."
8424,2021-12-30T18:02:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['택시', '기사', '업계', '공존', '카카오', '모빌리티', '상생', ...","['대리', '운전', '수수료', '개선', '이어', '새해', '상생', '플..."
8425,2021-12-31T14:03:00,https://news.naver.com/main/read.naver?mode=LS...,IT/과학,"['새로운', '골프', '시장', '개척', '게임', '체인', '카카오', '...","['코로나', '사태', '계기', '골프', '산업', '세대', '뿐', '아니..."


In [4]:
# 각 'tokenize_title'과 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [5]:
# title에 대한 LDA모델 적용
title_docs = []
for doc in df['tokenize_title']:
  title_docs.append(str_list(doc))
title_docs

[['네이버', '카카오', '추천', '뉴스', '사실', '여부', '보다', '많은', '클릭', '받을', '기사', '골라'],
 ['주식', '부호', '클럽', '김범수', '카카오', '의장', '순위', '껑충'],
 ['클럽', '카카오', '김범수', '늘', '고', '아모레', '경배', '폭락'],
 ['테크', '체인', '네이버', '보다', '카카오', '일상', '바꿀', '기업', '설문', '결과'],
 ['작년', '매출', '신기록', '쓴', '카카오', '엔씨', '올해', '좋다'],
 ['큰', '선다', '크래프톤', '카카오', '형제', '출격'],
 ['카뱅', '케뱅', '새해', '테크', '핀', '공격', '경영', '빅뱅'],
 ['기다리면', '무료', '재미', '본', '카카오', '영상', '반대', '유료'],
 ['낡은', '규', '제혁', '파', '된', '녹슨', '칼', '네이버', '카카오', '규제', '하겠다는', '정부'],
 ['땅', '좁다', '플라잉', '카', '띄우겠다는', '카카오', '독점', '깰까'],
 ['카카오', '손잡고', '세대', '공략'],
 ['카카오', '페이', '디지털', '손해', '보험사', '예비', '신청'],
 ['카카오', '페이', '디지털', '손해', '보험사', '예비', '신청'],
 ['카카오', '페이', '보험사', '설립', '속도', '예비', '신청', '완료'],
 ['카카오', '페이', '디지털', '손해', '보험사', '예비', '신청'],
 ['카카오', '페이', '디지털', '손해', '보험사', '설립', '예비', '신청'],
 ['카카오', '페이', '디지털', '설립', '예비', '신청', '완료'],
 ['카카오', '페이', '디지털', '손해', '보험사', '예비', '신청', '완료'],
 ['카카오', '페이', '디지털', '예비', '신청', '하반기', '출범', 

In [6]:
id2word_title = corpora.Dictionary(title_docs)
print(id2word_title)

Dictionary<7955 unique tokens: ['골라', '기사', '네이버', '뉴스', '많은']...>


In [7]:
for value in id2word_title:
  print(value, id2word_title[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2955 사는게
2956 나무
2957 연합군
2958 맞붙는
2959 안은
2960 첫날
2961 품
2962 붙자
2963 탔다
2964 남녀
2965 많고
2966 양대
2967 여직원
2968 임금
2969 적어
2970 몰아주나
2971 썰
2972 직
2973 듯이
2974 보란
2975 머
2976 번가
2977 포인트
2978 피자
2979 섯거
2980 기다려
2981 손잡은
2982 점화
2983 태운
2984 디쉬
2985 칼날
2986 홈쇼핑
2987 뭔가
2988 뭘
2989 스트리트
2990 있겠니
2991 서학
2992 열전
2993 근로
2994 쉽게
2995 유적지
2996 밀크
2997 블록
2998 사이드
2999 만났다
3000 노려
3001 베드
3002 테스트
3003 펫
3004 노
3005 믹스
3006 뉴욕증시
3007 훈풍
3008 부활
3009 독보
3010 잘나가네
3011 전세계
3012 선전
3013 나스닥
3014 주인공
3015 어떨까
3016 치솟은
3017 만발
3018 질렀다
3019 스테이지
3020 신한은행
3021 파이브
3022 했어요
3023 돈나무
3024 웃는
3025 선착순
3026 한경
3027 쟁
3028 위반
3029 혐의
3030 협상
3031 밀고
3032 바라본다
3033 들썩
3034 소문
3035 정도
3036 교보
3037 추정
3038 가능했나
3039 개강
3040 공매
3041 아카데미
3042 온비드
3043 캠코
3044 건의
3045 국토부
3046 제출
3047 철회
3048 대책
3049 하
3050 도대체
3051 몇개
3052 수십
3053 챙긴다
3054 파는
3055 거위
3056 낳는
3057 막
3058 팬덤
3059 하이브
3060 압박
3061 촉구
3062 모멘텀
3063 유예
3064 지그재그
3065 발굴
3066 신인
3067 개국
3068 방영
3069 단숨
3070 

In [8]:
corpus_TDM = []
for doc in title_docs:
  result = id2word_title.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1)],
 [(10, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)],
 [(10, 1), (12, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)],
 [(2, 1),
  (6, 1),
  (10, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1)],
 [(10, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)],
 [(10, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)],
 [(30, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)],
 [(10, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1)],
 [(2, 1),
  (10, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1)],
 [(10, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1)],
 [(10, 1), (74, 1), (75, 1), (76, 1)],
 [(10, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1)],
 [(10, 1), (77, 1), (78,

In [9]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_title, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.027*"김범수" + 0.024*"의장" + 0.019*"버전" + 0.016*"활동" + 0.013*"기능" + 0.010*"영업" + 0.009*"게임" + 0.008*"공정위" + 0.008*"주차장" + 0.007*"조사"
1 : 0.037*"미래" + 0.018*"게임" + 0.017*"대표" + 0.012*"출연" + 0.012*"이니셔티브" + 0.011*"센터" + 0.010*"올인" + 0.009*"먹거리" + 0.008*"증가" + 0.007*"이재용"
2 : 0.012*"반등" + 0.012*"스타트업" + 0.012*"투자" + 0.010*"펀드" + 0.009*"본격" + 0.009*"네이버" + 0.008*"카드" + 0.008*"출사표" + 0.008*"웹툰" + 0.008*"대감"
3 : 0.020*"집중" + 0.015*"하락" + 0.015*"코스피" + 0.013*"전환" + 0.009*"마감" + 0.009*"청약" + 0.009*"공개" + 0.009*"상승" + 0.008*"거래" + 0.008*"기관"
4 : 0.025*"상생" + 0.012*"포토" + 0.011*"카겜" + 0.010*"택시" + 0.009*"대리" + 0.009*"모빌리티" + 0.008*"골목" + 0.008*"헬" + 0.007*"상권" + 0.007*"규제"
5 : 0.012*"가맹" + 0.012*"제재" + 0.009*"패스" + 0.008*"손잡았다" + 0.008*"청년" + 0.007*"증발" + 0.006*"사과" + 0.006*"네이버" + 0.006*"인증" + 0.006*"기념"
6 : 0.024*"페이" + 0.017*"디지털" + 0.013*"상장" + 0.011*"신청" + 0.011*"모빌리티" + 0.010*"코스피" + 0.010*"구독" + 0.009*"사람" + 0.009*"투자" + 0.009*"인증서"
7 : 0.017*"예약" + 0.012*"골프" + 0.012*"네이버" + 0.011*"모집"

In [10]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(title 부분)
lda[corpus_TFIDF[0]]

[(0, 0.10747266),
 (1, 0.023886828),
 (2, 0.023884865),
 (3, 0.023885068),
 (4, 0.3441113),
 (5, 0.023891298),
 (6, 0.023904303),
 (7, 0.023885004),
 (8, 0.38119343),
 (9, 0.023885211)]

In [11]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.038071327), (1, 0.038075782), (2, 0.038080208), (3, 0.038072), (4, 0.03807187), (5, 0.35579622), (6, 0.33961093), (7, 0.038073283), (8, 0.038077246), (9, 0.03807116)]
[(0, 0.043762356), (1, 0.043765444), (2, 0.6061143), (3, 0.043763064), (4, 0.04376671), (5, 0.043767843), (6, 0.04376228), (7, 0.04377004), (8, 0.04376375), (9, 0.04376415)]
[(0, 0.19701219), (1, 0.031956915), (2, 0.37173164), (3, 0.0319555), (4, 0.20754403), (5, 0.0319694), (6, 0.031954527), (7, 0.03196089), (8, 0.031955265), (9, 0.031959604)]
[(0, 0.03747461), (1, 0.49784252), (2, 0.20236059), (3, 0.037474938), (4, 0.037474096), (5, 0.037474744), (6, 0.037475366), (7, 0.037474163), (8, 0.037474602), (9, 0.03747437)]
[(0, 0.041007977), (1, 0.15369461), (2, 0.2664408), (3, 0.041035224), (4, 0.04100942), (5, 0.04100954), (6, 0.041025724), (7, 0.041016616), (8, 0.041029327), (9, 0.2927307)]
[(0, 0.035585582), (1, 0.035586797), (2, 0.2608554), (3, 0.035586182), (4, 0.035585895), (5,

In [12]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(title)'] = dominant_topic[0]
  df.loc[i, 'topic_details(title)'] = dominant_topic[1]
df

,date,url,category,tokenize_title,tokenize_content,topic(title),topic_details(title)
0,2021-01-01T03:37:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['네이버', '카카오', '추천', '뉴스', '사실', '여부', '보다', '...","['네이버', '자체', '개발', '콘텐츠', '추천', '시스템', '어스', ...",8.0,0.381167
1,2021-01-01T08:00:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['주식', '부호', '클럽', '김범수', '카카오', '의장', '순위', '...","['코스피', '사상', '최고', '마감', '주식', '부자', '늘어', '고...",0.0,0.619700
2,2021-01-01T13:04:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['클럽', '카카오', '김범수', '늘', '고', '아모레', '경배', '폭락']","['코스피', '지수', '사상', '최고', '치', '마감', '하면서', '이...",4.0,0.737083
3,2021-01-01T13:21:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['테크', '체인', '네이버', '보다', '카카오', '일상', '바꿀', '...","['네이버', '카카오', '응답', '우리', '일상', '바꿀', '기대', '...",3.0,0.406932
4,2021-01-01T17:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['작년', '매출', '신기록', '쓴', '카카오', '엔씨', '올해', '좋다']","['증권사', '망치', '분석', '최대', '실적', '예고', '카카오', '...",0.0,0.504204
...,...,...,...,...,...,...,...
8422,2021-12-30T14:49:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['뉴스', '픽', '카카오', '지그재그', '패션', '임박']","['카카오', '온라인', '패션', '플랫폼', '지그재그', '자체', '브랜드...",5.0,0.466427
8423,2021-12-30T16:33:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['상생', '외친', '카카오', '모빌리티', '일', '성', '아닌', '지...","['업계', '의견', '수렴', '해', '장기', '상생', '구현', '목표'...",4.0,0.693250
8424,2021-12-30T18:02:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['택시', '기사', '업계', '공존', '카카오', '모빌리티', '상생', ...","['대리', '운전', '수수료', '개선', '이어', '새해', '상생', '플...",4.0,0.750366
8425,2021-12-31T14:03:00,https://news.naver.com/main/read.naver?mode=LS...,IT/과학,"['새로운', '골프', '시장', '개척', '게임', '체인', '카카오', '...","['코로나', '사태', '계기', '골프', '산업', '세대', '뿐', '아니...",9.0,0.404679


In [13]:
# 토픽별 비중 확인
df['topic(title)'].unique()

array([8., 0., 4., 3., 7., 6., 2., 1., 9., 5.])

In [14]:
df['topic(title)'].value_counts()

4.0    1050
1.0     979
6.0     957
0.0     910
8.0     864
9.0     864
7.0     850
3.0     825
2.0     605
5.0     523
Name: topic(title), dtype: int64

In [15]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [16]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<43439 unique tokens: ['가짜', '각', '갖지', '같은', '개발']...>


In [17]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
38439 태권브이
38440 특공대
38441 형상
38442 부당하단
38443 빼놓고
38444 늘리라
38445 뽑아라
38446 뽑으라
38447 실직자
38448 쏟아졌었다
38449 쟁의
38450 해직
38451 달려든
38452 벌어졌기
38453 줬는지에
38454 치켜세웠던
38455 심신
38456 내려가면
38457 모자라다
38458 불공평하다는
38459 주물
38460 힘써야
38461 꿨던
38462 내려놓기로
38463 뺏는다는
38464 잊히고
38465 잡다가
38466 국부
38467 자금지원
38468 구준엽
38469 그라피티
38470 나연
38471 낙서
38472 리듬
38473 빌리지
38474 서태지
38475 아트웍
38476 와와
38477 팝아트
38478 현진영
38479 두려워하고
38480 떼기에도
38481 몸살
38482 버리기로
38483 각주
38484 따져서
38485 딱딱하지만
38486 묶여있어서
38487 민법
38488 바뀌어도
38489 벌칙
38490 부르지만
38491 살펴봐
38492 용어인데요
38493 유사해요
38494 장황하게
38495 재미없고
38496 줍의
38497 풀리기
38498 맷집
38499 바꾸려하다
38500 부르겠다고
38501 빚기
38502 뽑자마자
38503 알려져있
38504 전관
38505 줬어야
38506 휘청대고
38507 내렸던
38508 나주시
38509 되어줄지는
38510 바꿀까
38511 비보
38512 술렁이고
38513 싸늘하기만
38514 족벌
38515 개점휴업
38516 뛰어넘고자
38517 부침에서
38518 시급해
38519 익숙함을
38520 친근함
38521 특권
38522 합쳤으며
38523 그려내면서
38524 나날
38525 담아내기
38526 담아내야
38527 망해
38528 보내던
38529 손수호
38530 심성
38531 어린아이로
38

In [18]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.004*"안산시" + 0.003*"파크" + 0.003*"보험" + 0.003*"착공" + 0.002*"이프" + 0.002*"안산" + 0.002*"면적" + 0.002*"캠퍼스" + 0.002*"데이터" + 0.002*"한양대"
1 : 0.002*"" + 0.002*"대차" + 0.001*"병원" + 0.001*"종로구" + 0.001*"분당" + 0.001*"매거진" + 0.001*"남편" + 0.001*"결혼" + 0.001*"손보" + 0.001*"보험"
2 : 0.001*"퍼포먼스" + 0.000*"대출" + 0.000*"신용" + 0.000*"불스원" + 0.000*"예스" + 0.000*"백신" + 0.000*"톤" + 0.000*"접종" + 0.000*"예방접종" + 0.000*"청과"
3 : 0.004*"안테나" + 0.003*"시청자" + 0.002*"예능" + 0.002*"드라마" + 0.002*"칼텍스" + 0.002*"채용" + 0.001*"아티스트" + 0.001*"대결" + 0.001*"장애인" + 0.001*"중구"
4 : 0.005*"택시" + 0.005*"모빌리티" + 0.005*"센터" + 0.003*"이니셔티브" + 0.003*"서비스" + 0.003*"상생" + 0.003*"리테일" + 0.002*"미래" + 0.002*"가맹" + 0.002*"렌터카"
5 : 0.005*"공매도" + 0.001*"딜카" + 0.001*"클라우드" + 0.001*"카셰어링" + 0.001*"엔터프라이즈" + 0.001*"중계" + 0.001*"캐피탈" + 0.001*"공공" + 0.000*"오픈소스" + 0.000*"장례식"
6 : 0.007*"게임" + 0.003*"페이" + 0.003*"대표" + 0.003*"주가" + 0.003*"네이버" + 0.002*"규제" + 0.002*"매출" + 0.002*"콘텐츠" + 0.002*"증권" + 0.002*"금융"
7 : 0.008*"캘린더" + 0.007*"메타" + 0.004*"

In [20]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(content 부분)
lda[corpus_TFIDF[0]]

[(0, 0.015528903),
 (1, 0.015515649),
 (2, 0.015515305),
 (3, 0.015558229),
 (4, 0.5121763),
 (5, 0.015524659),
 (6, 0.36363426),
 (7, 0.015515914),
 (8, 0.015515401),
 (9, 0.015515403)]

In [21]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.013295212), (1, 0.013294504), (2, 0.013294425), (3, 0.013294683), (4, 0.02922169), (5, 0.013294506), (6, 0.86442095), (7, 0.013294965), (8, 0.013294489), (9, 0.013294574)]
[(0, 0.0126101645), (1, 0.01260992), (2, 0.012609844), (3, 0.012610022), (4, 0.012640251), (5, 0.012609892), (6, 0.8864799), (7, 0.012610116), (8, 0.012609891), (9, 0.012609979)]
[(0, 0.0145373875), (1, 0.01453614), (2, 0.014534941), (3, 0.014536381), (4, 0.15736094), (5, 0.014535082), (6, 0.7263536), (7, 0.01453531), (8, 0.014535005), (9, 0.01453524)]
[(0, 0.010022165), (2, 0.010166599), (3, 0.17741963), (4, 0.15244943), (6, 0.4377702), (7, 0.17228504)]
[(0, 0.0134189), (1, 0.013409412), (2, 0.013409308), (3, 0.013409688), (4, 0.013436668), (5, 0.013409436), (6, 0.8792708), (7, 0.013409499), (8, 0.013410482), (9, 0.013415849)]
[(6, 0.9163335)]
[(4, 0.25706515), (6, 0.6665625)]
[(4, 0.10045003), (6, 0.82617944)]
[(4, 0.06539105), (6, 0.8643065)]
[(3, 0.060071208), (6, 0.7609

In [22]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(content)'] = dominant_topic[0]
  df.loc[i, 'topic_details(content)'] = dominant_topic[1]
df

,date,url,category,tokenize_title,tokenize_content,topic(title),topic_details(title),topic(content),topic_details(content)
0,2021-01-01T03:37:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['네이버', '카카오', '추천', '뉴스', '사실', '여부', '보다', '...","['네이버', '자체', '개발', '콘텐츠', '추천', '시스템', '어스', ...",8.0,0.381167,4.0,0.512319
1,2021-01-01T08:00:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['주식', '부호', '클럽', '김범수', '카카오', '의장', '순위', '...","['코스피', '사상', '최고', '마감', '주식', '부자', '늘어', '고...",0.0,0.619700,6.0,0.906654
2,2021-01-01T13:04:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['클럽', '카카오', '김범수', '늘', '고', '아모레', '경배', '폭락']","['코스피', '지수', '사상', '최고', '치', '마감', '하면서', '이...",4.0,0.737083,6.0,0.874114
3,2021-01-01T13:21:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"['테크', '체인', '네이버', '보다', '카카오', '일상', '바꿀', '...","['네이버', '카카오', '응답', '우리', '일상', '바꿀', '기대', '...",3.0,0.406932,6.0,0.709322
4,2021-01-01T17:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"['작년', '매출', '신기록', '쓴', '카카오', '엔씨', '올해', '좋다']","['증권사', '망치', '분석', '최대', '실적', '예고', '카카오', '...",0.0,0.504204,6.0,0.837885
...,...,...,...,...,...,...,...,...,...
8422,2021-12-30T14:49:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['뉴스', '픽', '카카오', '지그재그', '패션', '임박']","['카카오', '온라인', '패션', '플랫폼', '지그재그', '자체', '브랜드...",5.0,0.466427,6.0,0.766006
8423,2021-12-30T16:33:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['상생', '외친', '카카오', '모빌리티', '일', '성', '아닌', '지...","['업계', '의견', '수렴', '해', '장기', '상생', '구현', '목표'...",4.0,0.693250,4.0,0.719914
8424,2021-12-30T18:02:00,https://news.naver.com/main/read.naver?mode=LS...,경제,"['택시', '기사', '업계', '공존', '카카오', '모빌리티', '상생', ...","['대리', '운전', '수수료', '개선', '이어', '새해', '상생', '플...",4.0,0.750366,4.0,0.789598
8425,2021-12-31T14:03:00,https://news.naver.com/main/read.naver?mode=LS...,IT/과학,"['새로운', '골프', '시장', '개척', '게임', '체인', '카카오', '...","['코로나', '사태', '계기', '골프', '산업', '세대', '뿐', '아니...",9.0,0.404679,6.0,0.743718


In [23]:
# 토픽별 비중 확인
df['topic(content)'].unique()

array([4., 6., 3., 1., 5., 0., 7.])

In [24]:
df['topic(content)'].value_counts()

6.0    4935
4.0    3398
5.0      29
3.0      28
1.0      24
0.0       8
7.0       5
Name: topic(content), dtype: int64

In [25]:
# CSV 파일로 저장
csv_filename = "/content/drive/MyDrive/2023년 10월-11월 프로젝트/LDA모델 적용/news_2021.csv"
df.to_csv(csv_filename, index=False)